<a href="https://colab.research.google.com/github/Swapn2/PyTorch/blob/main/hyperparameter_tunning_bayesian_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.1 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [ ]:
import pandas as pd

In [ ]:
# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import numpy as np
cols_with_missing_vals = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0,np.nan)
df.fillna(df.mean(),inplace=True)

print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [ ]:
x= df.iloc[:,:-1]
y= df.iloc[:,-1]

In [ ]:
x

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.00000,180.000000,32.9,0.171,63
764,2,122.0,70.0,27.00000,155.548223,36.8,0.340,27
765,5,121.0,72.0,23.00000,112.000000,26.2,0.245,30
766,1,126.0,60.0,29.15342,155.548223,30.1,0.349,47


In [ ]:
y

,Outcome
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =.3, random_state= 42)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train.shape

(537, 8)

In [ ]:
x_test.shape

(231, 8)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def objective(trial):
  #  sugesting the values of the hyperparameters search space
  n_estimators = trial.suggest_int('n_estimators', 50 ,200)
  max_depth = trial.suggest_int('max_depth' ,3,20)

  #  creating the randomforestclassifier with suggested hyperparameters
  model = RandomForestClassifier(
      n_estimators= n_estimators,
      max_depth= max_depth,
      random_state=42
  )
  score = cross_val_score(model , x_train , y_train,cv = 3 , scoring= 'accuracy').mean()
  return score



In [ ]:
study = optuna.create_study(direction = 'maximize', sampler = optuna.samplers.TPESampler())
study.optimize(objective, n_trials= 50)

[I 2025-07-19 05:45:34,888] A new study created in memory with name: no-name-d0500a42-ddad-4cd8-90d1-29de859e0059
[I 2025-07-19 05:45:35,506] Trial 0 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 134, 'max_depth': 3}. Best is trial 0 with value: 0.7560521415270017.
[I 2025-07-19 05:45:36,154] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 132, 'max_depth': 6}. Best is trial 1 with value: 0.7672253258845437.
[I 2025-07-19 05:45:36,544] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 73, 'max_depth': 14}. Best is trial 1 with value: 0.7672253258845437.
[I 2025-07-19 05:45:37,088] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 109, 'max_depth': 18}. Best is trial 3 with value: 0.7746741154562384.
[I 2025-07-19 05:45:37,775] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 137, 'max_depth': 9}. Best is trial 3 with value: 0.77467411

In [ ]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 115, 'max_depth': 16}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [ ]:
# 1. Optimization History
plot_optimization_history(study).show()

# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

# 3. Slice Plot
plot_slice(study).show()

# 4. Contour Plot
plot_contour(study).show()

# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-07-19 05:50:28,143] A new study created in memory with name: no-name-b3c1af42-ecec-44ac-a082-db009c851d56
[I 2025-07-19 05:50:28,689] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 110, 'max_depth': 14}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-07-19 05:50:29,735] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 200, 'max_depth': 17}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-19 05:50:30,203] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 89, 'max_depth': 8}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-19 05:50:31,272] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 144, 'max_depth': 7}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-19 05:50:32,651] Trial 4 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 196, 'max_depth': 9}. Best is trial 1 with value: 0.77467411

In [ ]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 131, 'max_depth': 12}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [ ]:
# 1. Optimization History
plot_optimization_history(study).show()

# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

# 3. Slice Plot
plot_slice(study).show()

# 4. Contour Plot
plot_contour(study).show()

# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [ ]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [ ]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-07-19 05:51:14,543] A new study created in memory with name: no-name-1edd1fc9-f600-4d9d-bcdf-35d3fcf5451c
[I 2025-07-19 05:51:15,021] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-19 05:51:15,852] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-19 05:51:16,110] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-19 05:51:16,636] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-19 05:51:17,142] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [ ]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [ ]:
# 1. Optimization History
plot_optimization_history(study).show()

# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

# 3. Slice Plot
plot_slice(study).show()

# 4. Contour Plot
plot_contour(study).show()

# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [ ]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [ ]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-07-19 05:57:54,256] A new study created in memory with name: no-name-660ef1dc-d018-49cd-a396-43ae657545e7
[I 2025-07-19 05:57:54,308] Trial 0 finished with value: 0.7076350093109869 and parameters: {'classifier': 'SVM', 'C': 6.296946799501437, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.7076350093109869.
[I 2025-07-19 05:57:54,365] Trial 1 finished with value: 0.7076350093109869 and parameters: {'classifier': 'SVM', 'C': 15.484423613586767, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7076350093109869.
[I 2025-07-19 05:57:55,904] Trial 2 finished with value: 0.7374301675977654 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 77, 'learning_rate': 0.05585415805296896, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.7374301675977654.
[I 2025-07-19 05:57:56,069] Trial 3 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 27.827301732856117, 

In [ ]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.15476820186698861, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.707635,2025-07-19 05:57:54.261693,2025-07-19 05:57:54.308752,0 days 00:00:00.047059,6.296947,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.707635,2025-07-19 05:57:54.310493,2025-07-19 05:57:54.365512,0 days 00:00:00.055019,15.484424,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.737430,2025-07-19 05:57:54.371389,2025-07-19 05:57:55.904758,0 days 00:00:01.533369,NaN,NaN,GradientBoosting,NaN,NaN,0.055854,8.0,3.0,4.0,77.0,COMPLETE
3,3,0.785847,2025-07-19 05:57:55.905892,2025-07-19 05:57:56.069414,0 days 00:00:00.163522,27.827302,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.763501,2025-07-19 05:57:56.070749,2025-07-19 05:57:56.114732,0 days 00:00:00.043983,6.158562,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-07-19 05:58:27.167939,2025-07-19 05:58:27.192867,0 days 00:00:00.024928,0.148145,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-07-19 05:58:27.193397,2025-07-19 05:58:27.258912,0 days 00:00:00.065515,9.141579,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.724395,2025-07-19 05:58:27.259623,2025-07-19 05:58:27.289603,0 days 00:00:00.029980,0.268967,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.770950,2025-07-19 05:58:27.290201,2025-07-19 05:58:27.324668,0 days 00:00:00.034467,0.211422,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,80
GradientBoosting,10
RandomForest,10


In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.743017
RandomForest,0.764246
SVM,0.776769


In [ ]:
# 1. Optimization History
plot_optimization_history(study).show()

# 3. Slice Plot
plot_slice(study).show()

# 5. Hyperparameter Importance
plot_param_importances(study).show()